In [2]:
import pandas as pd

customers_df = pd.read_csv('../data/hm/customers.csv')
articles_df = pd.read_csv('../data/hm/articles.csv', dtype={'article_id': str})

print('hi')

article_df = articles_df[['article_id', 'prod_name', 'product_type_name']]
customers_df = customers_df[['customer_id', 'age']]
transactions_df = pd.read_csv('../data/hm/transactions_train.csv', dtype={'article_id': str})

print('hi!')

df = pd.merge(transactions_df, article_df, how='inner', on='article_id')
customer_count = df.customer_id.value_counts()
valid_customers = customer_count[5000:15000]
valid_customer_ids = set(valid_customers.index)
filtered_df = df[df.customer_id.isin(valid_customer_ids)]

print(len(filtered_df))

print('hi!!')

hi
hi!
2136454
hi!!


In [20]:
from tqdm import tqdm
tqdm.pandas()

raw_sequences = []
seq_labels = None

for customer_id, personal_df in tqdm(filtered_df.groupby('customer_id')):
    raw_sequences.append(list(personal_df.article_id.values))

100%|██████████| 10000/10000 [00:00<00:00, 17107.06it/s]


In [21]:
filtered_articles = filtered_df.article_id.unique()

In [22]:
item_list = []
filtered_articles_df = articles_df[article_df.article_id.isin(set(filtered_articles))]

for index, row in filtered_articles_df.iterrows():
    item_name = row.prod_name + ' - ' + row.product_type_name
    item_list.append(item_name)

In [23]:
from gensim.models import word2vec

word2vec_model = word2vec.Word2Vec(
    raw_sequences,
    vector_size=50,
    window=5,
    min_count=0
)
word2vec_model.save('../weights/word2vec.model')
item_embedding = [word2vec_model.wv[article_id] for article_id in filtered_articles]

In [24]:
raw_sequences = [' '.join(sequence) for sequence in raw_sequences]

In [25]:
sequence_df = pd.DataFrame(raw_sequences, columns=['sequence'])
sequence_df.to_csv('../data/hm/purchase_history.csv')

In [26]:
item_df = pd.DataFrame(item_list, columns=['name'])
item_df['article_id'] = filtered_articles
item_df.to_csv('../data/hm/items.csv')